In [1]:
import torch, csv, emoji
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from transformers import AutoModel, AutoTokenizer 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer

from NN import FILEPATH, read_csv, emojiDataset, toEmoji

Using cpu device


Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[[ 0.19230774 -0.1516456   0.08332738 ... -0.03129064 -0.09845931
  -0.13666552]
 [ 0.20112911 -0.18538275  0.09357507 ... -0.10974507 -0.13172388
  -0.18648902]
 [ 0.18441607 -0.22273619  0.05751167 ... -0.04333666 -0.17014931
  -0.15209252]
 ...
 [ 0.19139415 -0.16448398  0.12457529 ... -0.04692347 -0.14717954
  -0.22945866]
 [ 0.25487196 -0.14747491  0.04594442 ... -0.15331753 -0.11392393
  -0.24846049]
 [ 0.26149157 -0.09217862 -0.05390346 ... -0.05662335 -0.09407201
  -0.13251233]]


In [2]:
a = read_csv(FILEPATH)
print(a[0][-1], a[1][-1])

And we're back to our regularly scheduled not-Lanny months.\n\nDid you miss her? [1]


In [3]:
torch.cuda.is_available()

False

In [4]:
dataset = emojiDataset()

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[[ 0.19230774 -0.1516456   0.08332738 ... -0.03129064 -0.09845931
  -0.13666552]
 [ 0.20112911 -0.18538275  0.09357507 ... -0.10974507 -0.13172388
  -0.18648902]
 [ 0.18441607 -0.22273619  0.05751167 ... -0.04333666 -0.17014931
  -0.15209252]
 ...
 [ 0.19139415 -0.16448398  0.12457529 ... -0.04692347 -0.14717954
  -0.22945866]
 [ 0.25487196 -0.14747491  0.04594442 ... -0.15331753 -0.11392393
  -0.24846049]
 [ 0.26149157 -0.09217862 -0.05390346 ... -0.05662335 -0.09407201
  -0.13251233]]


In [5]:
data_loader = DataLoader(dataset=dataset, batch_size=16, shuffle=True,num_workers=2)
sentences, emojis, labels_classes = dataset.get_others()

# data_iter = iter(data_loader)
# next_data = data_iter.next()
# features, labels = next_data
# print(features, labels)

toemoji = toEmoji()
learn_rate = optim.Adam(toemoji.parameters(), lr=0.005)
loss_func = nn.MSELoss()
# loss_func = nn.NLLLoss()
epochs = 10

In [6]:
for i in range(epochs):
    for data in data_loader:
        features, labels = data
        toemoji.zero_grad()
        results = toemoji(features.float())
        # print(result)
        # print(results, labels)
        loss = loss_func(results, labels)
        # for result, label in zip(results, labels):
        #     loss += F.nll_loss(result, label) 
        loss.backward()
        learn_rate.step()
    print(loss)

tensor(0.5725, grad_fn=<MseLossBackward0>)
tensor(0.7376, grad_fn=<MseLossBackward0>)
tensor(0.4558, grad_fn=<MseLossBackward0>)
tensor(0.2971, grad_fn=<MseLossBackward0>)
tensor(0.1420, grad_fn=<MseLossBackward0>)
tensor(0.0126, grad_fn=<MseLossBackward0>)
tensor(0.0490, grad_fn=<MseLossBackward0>)
tensor(0.1245, grad_fn=<MseLossBackward0>)
tensor(0.0653, grad_fn=<MseLossBackward0>)
tensor(0.2061, grad_fn=<MseLossBackward0>)


In [7]:
K = 2

with torch.no_grad():
    for i in range(len(sentences)) :
        feature, label = dataset[i]
        result = toemoji(feature.float())
        print(f'Sentence: {sentences[i]}\nOriginal labels: {emojis[i][0]}, output labels: {result[0]}\n') 
        # original_str, result_str = '', ''

        # top_k = (-result).argsort()[:K]
        # for top_index in top_k:
        #     result_str += labels_classes[top_index]
        
        # original_str = ''.join(emojis[i])
        
        # print(f'Sentence: {sentences[i]}\nOriginal labels: {original_str}, output labels: {result_str}\n')
        

Sentence: Honestly my biggest worry is we haven't heard any rumors about RB.
Original labels: 0, output labels: -0.04248300939798355

Sentence: CHO is just so good, what he did to mari 😫
Original labels: 0, output labels: -0.03445057570934296

Sentence: this is an example tweet
Original labels: 1, output labels: 1.0959835052490234

Sentence: one month til school starts 😞😞
Original labels: 0, output labels: 0.05233239382505417

Sentence: The Brazilian people demonstrating in favor of the auditable vote
Original labels: 1, output labels: 0.7027367949485779

Sentence: i can’t believe that whole high hong situation happened 😭
Original labels: 0, output labels: -0.008939778432250023

Sentence: tell em cut the check!
Original labels: 0, output labels: 0.11702284961938858

Sentence: THANKS I HATE IT AND NOW HAVE TO SHARE IT WITH EVERYONE
Original labels: 0, output labels: -0.006699202582240105

Sentence: W
Original labels: 2, output labels: 0.96123868227005

Sentence: Bothered¿
Original label